#### Thread vs. Process
Thread shares the __address space__ so can access the same data. 

Both have its own __PC, context switch, PCB__. 

#### Critical Section
__Mutual exclusion__ only one thread in critical section  
__Progress__ if we can make progress, then we must make  
__No starvation__ limited number of times other threads can enter CS before the thread is granted access

#### Examples

``` c
My_work(id_t id) { /* id can be 0 or 1 */
    ...
    while (turn != id) {
         /* cs */
        turn = 1 - id;
    }
    ...
}
```

__Progress__ is not satisfied, as two workers must switch between each other, even one worker can do multiple rounds

``` c
My_work(id_t id) { /* id can be 0 or 1 */
    ...
    while (flag[1 - id]); /* entry section */
    flag[id] = true; /* indicate entering CS */
    /* cs */
    flag[id] = false; /* exit section */
    ...
}
```

__Mutual exclusion__ is not satisfied, each thread executes while statement, finds flag set to false, or each thread sets own flag to true and enters CS. 

#### Peterson's Algorithm

``` c
int turn;
int flag[2];

My_work(id_t id) {
    ...
    flag[id] = true;
    turn = id;
    while (turn == id && flag[1-id]); 
    /* cs */
    flag[id] = false;
    ...
}
```

### Test and Set 
``` c
boolean test_and_set(bool *lock) {
    bool old = *lock;
    *lock = true;
    return old;
}

boolean test_and_set(bool *lock) {
    if (*lock == false) {
        *lock = true;
        return false;
    } else {
        return true;
    }
}
```

Using this, can build a __spin lock__

When false, we know that we've acquired it
``` c
void acquire(bool *lock) {
    while (test_and_set(lock));
}
```

To release, set to false
``` c
void release(bool *lock) {
    *lock = false;
}
```
However, this requires busy waiting, continually executes while loop in `acquire`, consumes CPU cycles

### Higher Level Abstractions

#### Semaphor 
``` c
wait(sem) {
    while (sem <= 0);
    sem--;
}
signal(sem) {
    sem++;
}
```
semaphore value represents the number of threads that can pass through wait before it blocks. 

#### Example
Guarantee a1 happens before b2 and b1 happens before a2. 
``` c
sem_t sem = -1;
a1
wait(sem);
signal(sem);
wait(sem);
a2
signal(sem);

b1
signal(sem);
wait(sem);
b2
signal(sem);
```

Note that `sem` cannot be 0, otherwise it's impossible. However, this will ends up with 1, which cannot be repeated. 

``` c
sem_t semA / semB = 0;
a1 / b1
signal(semB) / signal(semA);
wait(semA) / wait(semB);
a2 / b2
```

#### Conditional Variables
```c
cv_wait(cv *cv, lock *lock) // release lock, wait, re-acquires lock before return
cv_signal(cv *cv, lock *lock) // wake one enqueued thread
cv_broadcast(cv *cv, lock *lock) // wakes all enqueued threads
```

CV always used together with locks

the lock protects the shared data that is modified and tested when deciding CV operations.

General usage

``` c
lock_acquire(lock);
while (condition not true) {
    cv_wait(cond, lock)
}
... // do stuff
cv_signal(cond); // or cv_broadcast(cond)
lock_release(lock);
```

__Signal__ the OS scheduler will randomly pick one  
__Broadcast__ all the threads wake up

#### Reader Writer Problem

__Writer__ 
``` c
lock(&mutex);
while (readcount || writing) {
    cv_wait(&turn, &mutex)
}
writing++;
unlock(&mutex);

// Write

lock(&mutex);
writing--;
cv_broadcast(&turn);
unlock(&mutex);
```

__Reader__
``` c
lock(&mutex);
while (writing) {
    cv_wait(&turn, &mutex);
}
readcount++;
unlock(&mutex);

// Read

lock(&mutex);
readcount--;
cv_signal(&turn);
unlock(&mutex);
```

#### Reader / Writer with Semaphores
```
int readcount = 0;
sem mutex = 1;
sem w_or_r = 1;
```
__Writer__
```c
wait(w_or_r);
// Write
signal(w_or_r);
```

__Reader__
```c
wait(mutex);
readcount++;

if (readcount == 1) {
    wait(w_or_r);
}
signal(mutex);
//Read
wait(mutex);
readcount--;

if (readcount == 0) {
    signal(w_or_r);
}
signal(mutex);
```